In [46]:
import geopandas as gpd
import pandas as pd
import numpy as np
import pickle
from shapely import Point, Polygon
from shapely.ops import nearest_points
from rtree import index

In [47]:
# Import and subset block data

blocks = gpd.read_file('blocks/tl_2020_06037_tabblock20.shp')
blocknumbers = np.array(blocks["GEOID20"],dtype=str)
blockboundaries = np.array(blocks["geometry"])

nodes_edges_weighted = pd.read_pickle('graph_centrality_codes/nodes_edges_weighted.pickle')
B_matrix_weighted, node_coordinates_weighted = nodes_edges_weighted
Hillside_NodeCoordinate = node_coordinates_weighted[:,0:2]

In [48]:
# EDA

print(len(blocknumbers))
print(type(blocknumbers[0]))
print(len(blockboundaries))
print(type(blockboundaries[0]))

print(len(Hillside_NodeCoordinate))
print(type(Hillside_NodeCoordinate))

91626
<class 'numpy.str_'>
91626
<class 'shapely.geometry.polygon.Polygon'>
50124
<class 'numpy.ndarray'>


In [49]:
# INITIAL APPROACH: Using for loops to check each node coordinate against each block boundary (very slow)

# Node_Block = np.empty((0, 4), dtype=object)

# if len(blockboundaries) != len(blocknumbers):
#     print("Error: Length of blockboundaries and blocknumbers do not match")

# for i in range(4):
#     for j in range(len(blockboundaries)):
#         if blockboundaries[j].contains(Point(Hillside_NodeCoordinate[i])):
#             row = [Hillside_NodeCoordinate[i][0], Hillside_NodeCoordinate[i][1], blocknumbers[j], blockboundaries[j]]
#             Node_Block = np.append(Node_Block, [row], axis=0)
#         else:
#             continue

In [50]:
# SECOND APPROACH: Using list comprehension to check each node coordinate against each block boundary (equally very slow)

# Node_Block = []

# if len(blockboundaries) != len(blocknumbers):
#     print("Error: Length of blockboundaries and blocknumbers do not match")

# Node_Block = [[Hillside_NodeCoordinate[i], blocknumbers[j], blockboundaries[j]] for i in range(4) for j in range(len(blockboundaries)) if blockboundaries[j].contains(Point(Hillside_NodeCoordinate[i]))]

In [51]:
# FINAL APPROACH: Using R-tree index to check each node coordinate against each block boundary (significantly faster - should be scalable to even larger datasets)

if len(blockboundaries) != len(blocknumbers):
    print("Error: Length of blockboundaries and blocknumbers do not match")

# Create an R-tree index
idx = index.Index()
for i, boundary in enumerate(blockboundaries):
    idx.insert(i, boundary.bounds)

In [52]:
Node_Block = []
Unidentified_Nodes = []

for i in range(len(Hillside_NodeCoordinate)):
    coord = Hillside_NodeCoordinate[i]
    possible_blocks = list(idx.intersection((coord[0], coord[1], coord[0], coord[1])))
    for block_idx in possible_blocks:
        if blockboundaries[block_idx].contains(Point(coord[0], coord[1])):
            Node_Block.append([i, blocknumbers[block_idx], blockboundaries[block_idx]])

Identified_Nodes = np.array([row[0] for row in Node_Block])

In [53]:
total_nodes_identified = len(Node_Block)

total_nodes_out_of_bounds = len(Hillside_NodeCoordinate) - total_nodes_identified

extracted_blocknumbers = np.array([item[1] for item in Node_Block])

unique_blocknumbers = np.unique(extracted_blocknumbers)

unique_blocknumbers_count = len(unique_blocknumbers)

print("Total nodes identified: " + str(total_nodes_identified), "\nNodes out of bounds: " + str(total_nodes_out_of_bounds), "\nUnique blocknumbers: " + str(unique_blocknumbers_count))

Total nodes identified: 50122 
Nodes out of bounds: 2 
Unique blocknumbers: 24415


In [54]:
# Make a list with the first element in Node_Block

Identified_Nodes_Index = [item[0] for item in Node_Block]

In [55]:
# Random check to see if the identified nodes are in the correct block

# Subset 10 random node indices from Identified_Nodes_Index

np.random.seed(123)
random_numbers = np.random.choice(Identified_Nodes_Index, 10, replace=False)

In [56]:
# Check if the random nodes are in the correct block

for i in random_numbers:
    if Node_Block[i][2].contains(Point(Hillside_NodeCoordinate[i][0], Hillside_NodeCoordinate[i][1])):
        print('Node coordinate ' + str(Node_Block[i][0]) + ' with coordinates ' + str(Hillside_NodeCoordinate[i]) + ' is in block ' + str(Node_Block[i][1]))
    else:
        print('ERROR: Node coordinate ' + str(Node_Block[i][0]) + ' with coordinates ' + str(Hillside_NodeCoordinate[i]) + ' is not in block ' + str(Node_Block[i][1]))

Node coordinate 41016 with coordinates [-118.16236153   34.07909363] is in block 060372016022009
Node coordinate 41192 with coordinates [-118.29844188   33.7563765 ] is in block 060372963002000
Node coordinate 34879 with coordinates [-118.26066232   34.06726872] is in block 060372083022000
Node coordinate 11598 with coordinates [-118.45655572   34.28640521] is in block 060371066491000
Node coordinate 13260 with coordinates [-118.44301205   34.0506783 ] is in block 060372655242001
Node coordinate 37428 with coordinates [-118.23298138   34.1082143 ] is in block 060371864041000
Node coordinate 29132 with coordinates [-118.29788057   34.09084238] is in block 060371912041001
Node coordinate 42225 with coordinates [-118.54530454   34.25458498] is in block 060371112041012
Node coordinate 30601 with coordinates [-118.28852464   34.10179367] is in block 060371891012007
Node coordinate 2263 with coordinates [-118.59290332   34.22820298] is in block 060371134252001


In [57]:
# Identify nodes that are not in any block

for i in range(len(Hillside_NodeCoordinate)):
    if i not in Identified_Nodes:
        Unidentified_Nodes.append(i)

print('Nodes without block placement: ' + str(Unidentified_Nodes))

[46025, 48626]


In [58]:
Hillside_NodeCoordinate

array([[-118.66764857,   34.17992453],
       [-118.66729721,   34.17746687],
       [-118.66691275,   34.1815081 ],
       ...,
       [-118.4079475 ,   33.97965507],
       [-118.22871315,   33.78572582],
       [-118.22876022,   33.78619129]])

In [59]:
for i in Unidentified_Nodes:
    closest_block = min(blockboundaries, key=lambda x: x.distance(Point(Hillside_NodeCoordinate[i][0], Hillside_NodeCoordinate[i][1])))
    closest_block_idx = np.where(blockboundaries == closest_block)
    closest_block_number = blocknumbers[closest_block_idx]
    print('Node coordinate ' + str(i) + ' with coordinates ' + str(Hillside_NodeCoordinate[i]) + ' is closest to block ' + str(closest_block_number))
    Node_Block.insert(i, [i, closest_block_number, closest_block])

Node coordinate 46025 with coordinates [-118.66781189   34.20678701] is closest to block ['060371344241013']
Node coordinate 48626 with coordinates [-118.66817614   34.18493116] is closest to block ['060371352052002']


In [62]:
total_nodes_identified = len(Node_Block)

print("Total nodes identified: " + str(total_nodes_identified))

Total nodes identified: 50124


In [63]:
# 060371352051001

In [64]:
with open('Node_Block.pkl', 'wb') as f:
    pickle.dump(Node_Block, f)